In [11]:
import sys
sys.path.append('../')
sys.path.append('../support/')
sys.path.append('../lung_segmentation/')
import os
import SimpleITK
from lung_separation import *
from ct_reader import *
from os.path import join, basename, isfile
from glob import glob
from scipy.ndimage import morphology
from skimage import filters
from skimage import exposure
from tqdm import tqdm
from skimage import morphology as skm
from skimage.morphology import watershed
from scipy.ndimage import label
from scipy.ndimage import generate_binary_structure
from skimage import measure
from multiprocessing import Pool
import pickle
from pure_ws_segmentation import *
import lung_separation_frontal
from numpy import *
# import warnings
# warnings.filterwarnings('ignore')

import SimpleITK as sitk
from paths import * 
from skimage.transform import resize

In [12]:
erroneus = sorted(list(set(pickle.load(open(join(PATH['STAGE_MASKS'], 'still_erroneus_ncrash'), 'rb')))))
erroneus = [join(PATH['STAGE_DATA'],err) for err in erroneus]

In [14]:
def operate(path, out_dir='STAGE_MASKS'):
    ct_scan = read_ct_scan(path)
    ct_scan_px = get_pixels_hu(ct_scan)

    ct_excluded = []
    for sl in (ct_scan_px):
        ct_excluded.append(exclude_lungs(sl))
#     with Pool(34) as pool:
#         ct_excluded = pool.map(exclude_lungs, ct_scan_px)

    # end = time.time()
    # print(end - start)

    lung_filter = asarray(ct_excluded)
    a128 = lung_filter.min()
    a255 = lung_filter.max()
    lung_filter[lung_filter==a128] = 0
    lung_filter[lung_filter==a255] = 1
    left, right = lung_separation_frontal.separate_lungs3d(lung_filter)
    save(join(join('/home/a.dobrenkii/Projects/Kaggle/DataScienceBowl2K17/data/NEW_STAGE/MASKS','FIXED'),basename(path)),left+2*right)

In [ ]:
for err in tqdm(erroneus):
    operate(err)


  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/102 [00:00<?, ?it/s]

finding start slice




  1%|▏         | 4/306 [00:00<00:08, 36.88it/s]

moving forward



  3%|▎         | 6/205 [00:00<00:03, 59.19it/s]

...


  0%|          | 0/102 [00:00<?, ?it/s]

finding start slice




  4%|▍         | 13/306 [00:00<00:02, 128.82it/s]

moving forward



 88%|████████▊ | 268/306 [02:15<00:42,  1.12s/it]